In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
import ipywidgets as widgets
from IPython.display import display
import plotly.express as px

In [3]:
import pandas as pd

# Laden der Datei in ein Pandas DataFrame
datei_path = r"C:\Users\Kunde\Desktop\BNT_Technikerschule\Dr. Movsessian\Projekt Windkraftanlage\Features_for_SHAP32rpm_repaierd.txt"
df = pd.read_csv(datei_path)

# Anzeigen der ersten Zeilen des DataFrames
df.tail()


# ...

,Cov1_1,Cov1_2,Cov1_4,Cov1_5,Cov1_7,Cov1_8,Cov1_10,Cov1_11,Cov2_2,Cov2_4,...,DC2_RPM,Wind_Direction,Temperature,Preciptation,Generator_on_off,Power_kW,Pitch_deg,Yaw,Azimuth,novelty_index
2922,1926.096405,22.077982,175.857296,-45.826991,50.419635,-196.514162,-19.894280,17.230319,106.262023,73.319511,...,32.025182,225.3840,2.175221,0.005802,1.0,-0.203927,-0.204647,10.02373,-35.567223,19.300004
2923,1968.053957,1.849336,237.957362,-40.600944,37.026843,-172.434916,-17.566236,12.396130,92.356970,53.739386,...,32.024936,225.4025,2.175308,0.005802,1.0,-0.203776,-0.166080,10.02100,-35.075370,18.866672
2924,1878.377866,17.217453,313.894413,-33.844109,56.978583,-157.683324,-14.607295,7.955903,96.110466,57.960094,...,32.025416,224.6777,2.154097,0.005802,1.0,-0.202933,-0.446031,10.02232,-27.235961,19.676828
2925,1883.592661,56.197118,452.159911,-29.978124,108.109518,-139.086048,6.094632,11.027323,106.358605,72.326554,...,32.026613,220.8360,2.157872,0.005802,1.0,-0.202147,-0.475663,10.02020,6.454405,19.960807
2926,1888.385538,48.195217,431.133481,-29.426011,87.764201,-129.770940,21.244808,12.426216,108.380538,70.841707,...,32.031763,213.0807,2.180657,0.005802,1.0,-0.202552,-0.527526,10.03005,104.349985,22.522516


# Anzeige in Tabs (Jupyter Widgets)
## Mit Jupyter Widgets können Sie die Spalten in Tabs aufteilen, um die Anzeige zu erleichter

In [4]:
def display_df_in_tabs(df, chunk_size=10):
    num_chunks = (len(df.columns) + chunk_size - 1) // chunk_size
    for i in range(num_chunks):
        start = i * chunk_size
        end = min((i + 1) * chunk_size, len(df.columns))
        chunk = df.iloc[:, start:end]
        tab = widgets.Tab(children=[widgets.Output(layout=widgets.Layout(width='auto')) for _ in range(len(chunk.columns))],
                          titles=list(chunk.columns))
        for j, col in enumerate(chunk.columns):
            with tab.children[j]:
                display(chunk[col])
        display(tab)

display_df_in_tabs(df)



In [5]:
df_2=df
print(df_2["Generator_on_off"].value_counts())


Generator_on_off
1.000000    2127
0.000000      31
0.888542       1
0.807813       1
0.701042       1
0.854167       1
0.658333       1
0.752083       1
0.475521       1
0.744271       1
0.857812       1
0.846354       1
Name: count, dtype: int64


In [6]:
df.columns

Index(['Cov1_1', 'Cov1_2', 'Cov1_4', 'Cov1_5', 'Cov1_7', 'Cov1_8', 'Cov1_10',
       'Cov1_11', 'Cov2_2', 'Cov2_4', 'Cov2_5', 'Cov2_7', 'Cov2_8', 'Cov2_10',
       'Cov2_11', 'Cov4_4', 'Cov4_5', 'Cov4_7', 'Cov4_8', 'Cov4_10', 'Cov4_11',
       'Cov5_5', 'Cov5_7', 'Cov5_8', 'Cov5_10', 'Cov5_11', 'Cov7_7', 'Cov7_8',
       'Cov7_10', 'Cov7_11', 'Cov8_8', 'Cov8_10', 'Cov8_11', 'Cov10_10',
       'Cov10_11', 'Cov11_11', 'WindSpeed', 'DC1_RPM', 'DC2_RPM',
       'Wind_Direction', 'Temperature', 'Preciptation', 'Generator_on_off',
       'Power_kW', 'Pitch_deg', 'Yaw', 'Azimuth', 'novelty_index'],
      dtype='object')

In [7]:


# ✅ 1. Daten laden (angenommen, dein DataFrame heißt df)
# Falls du CSV-Daten hast, kannst du sie so laden: df = pd.read_csv("deine_datei.csv")

# ✅ 2. Wichtige Spalten auswählen
relevante_spalten = [
    "WindSpeed", "Wind_Direction", "Temperature", "Preciptation",  # Umweltbedingungen
    "DC1_RPM", "DC2_RPM", "Power_kW", "Pitch_deg", "Yaw", "Azimuth",  # Betriebsdaten
    "Generator_on_off", # Status 
]

# Falls "Cov1"-"Cov36" Sensordaten sind und du sie behalten willst:
cov_spalten = [col for col in df.columns if col.startswith("Cov")]

# Endgültige Spaltenliste
features = relevante_spalten + cov_spalten

# ✅ 3. Daten filtern
df = df[features]

# ✅ 4. Fehlende Werte behandeln (hier: Entfernen von Zeilen mit NaN)
df = df.dropna()

# ✅ 5. Skalierung der numerischen Werte (außer Generator_on_off, da 0/1)
scaler = MinMaxScaler()
numerische_spalten = [col for col in df.columns if col != "Generator_on_off"]
df[numerische_spalten] = scaler.fit_transform(df[numerische_spalten])

# ✅ 6. Ergebnis anzeigen
print(df.head())

# Optional: Daten speichern
# df.to_csv("vorbereitete_daten.csv", index=False)


   WindSpeed  Wind_Direction  Temperature  Preciptation   DC1_RPM   DC2_RPM  \
0   0.145542        0.838095     0.363770      0.953488  0.968860  0.946192   
1   0.112612        0.863321     0.359408      0.953488  0.970191  0.949021   
2   0.105434        0.851611     0.358454      0.953488  0.950558  0.930401   
3   0.104907        0.877472     0.355140      0.953488  0.914142  0.895160   
4   0.093385        0.853076     0.350759      0.953488  0.205302  0.779602   

   Power_kW  Pitch_deg       Yaw   Azimuth  ...    Cov7_7    Cov7_8   Cov7_10  \
0  0.078971   0.431931  0.980381  0.720479  ...  0.646088  0.486132  0.488080   
1  0.079062   0.489718  0.975601  0.969982  ...  0.615289  0.304033  0.458650   
2  0.079076   0.122621  0.976113  0.551947  ...  0.713952  0.634155  0.647971   
3  0.079094   0.048987  0.980301  0.400981  ...  0.602873  0.511900  0.432717   
4  0.078988   0.047906  0.983583  0.381388  ...  0.594061  0.424264  0.377281   

    Cov7_11    Cov8_8   Cov8_10   Cov8

In [8]:

from scipy.spatial.distance import mahalanobis

# ✅ 1. Kovarianzmatrix berechnen
cov_matrix = np.cov(df[numerische_spalten].values, rowvar=False)
cov_inv = np.linalg.inv(cov_matrix)  # Inverse der Kovarianzmatrix

# ✅ 2. Mittelwerte des Normalzustands berechnen
mean_values = df[numerische_spalten].mean().values  

# ✅ 3. Mahalanobis-Distanz für jede Zeile berechnen
df["Schadensindex"] = df[numerische_spalten].apply(lambda x: mahalanobis(x, mean_values, cov_inv), axis=1)

# ✅ 4. Schadensindex normalisieren (0-1 Skala)
df["Schadensindex"] = (df["Schadensindex"] - df["Schadensindex"].min()) / (df["Schadensindex"].max() - df["Schadensindex"].min())

# ✅ 5. Ergebnisse anzeigen
print(df[["Schadensindex"] + numerische_spalten].head())

# Optional: Daten speichern
# df.to_csv("schadensindex_daten.csv", index=False)


   Schadensindex  WindSpeed  Wind_Direction  Temperature  Preciptation  \
0       0.093362   0.145542        0.838095     0.363770      0.953488   
1       0.094993   0.112612        0.863321     0.359408      0.953488   
2       0.130778   0.105434        0.851611     0.358454      0.953488   
3       0.133835   0.104907        0.877472     0.355140      0.953488   
4       0.535795   0.093385        0.853076     0.350759      0.953488   

    DC1_RPM   DC2_RPM  Power_kW  Pitch_deg       Yaw  ...    Cov7_7    Cov7_8  \
0  0.968860  0.946192  0.078971   0.431931  0.980381  ...  0.646088  0.486132   
1  0.970191  0.949021  0.079062   0.489718  0.975601  ...  0.615289  0.304033   
2  0.950558  0.930401  0.079076   0.122621  0.976113  ...  0.713952  0.634155   
3  0.914142  0.895160  0.079094   0.048987  0.980301  ...  0.602873  0.511900   
4  0.205302  0.779602  0.078988   0.047906  0.983583  ...  0.594061  0.424264   

    Cov7_10   Cov7_11    Cov8_8   Cov8_10   Cov8_11  Cov10_10  Cov10

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# ✅ Zeilenindex als Ersatz für Zeitstempel
df["Index"] = df.index  

# ✅ Funktion zur Erstellung der Diagramme
def plot_schadensindex(typ):
    plt.figure(figsize=(8, 5))

    if typ == "Trend (Index)":
        plt.plot(df["Index"], df["Schadensindex"], color="red", label="Schadensindex")
        plt.xlabel("Datenindex")
        plt.ylabel("Schadensindex")
        plt.title("Schadensindex über die Datenreihenfolge")
        plt.legend()

    elif typ == "Windgeschwindigkeit":
        plt.scatter(df["WindSpeed"], df["Schadensindex"], alpha=0.5, color="blue")
        plt.xlabel("Windgeschwindigkeit (m/s)")
        plt.ylabel("Schadensindex")
        plt.title("Zusammenhang zwischen Windgeschwindigkeit und Schadensindex")

    elif typ == "Temperatur":
        plt.scatter(df["Temperature"], df["Schadensindex"], alpha=0.5, color="green")
        plt.xlabel("Temperatur (°C)")
        plt.ylabel("Schadensindex")
        plt.title("Einfluss der Temperatur auf den Schadensindex")

    plt.grid()
    plt.show()

# ✅ Dropdown-Widget für Auswahl
dropdown = widgets.Dropdown(
    options=["Trend (Index)", "Windgeschwindigkeit", "Temperatur"],
    value="Trend (Index)",
    description="Diagramm:",
)

# ✅ Anzeige aktualisieren, wenn Auswahl geändert wird
widgets.interactive(plot_schadensindex, typ=dropdown)


interactive(children=(Dropdown(description='Diagramm:', options=('Trend (Index)', 'Windgeschwindigkeit', 'Temp…